# 110. Basic Linear Regression

In [1]:
import numpy as np
import torch

# 1) Feed Forward 구현

- weight와 bias를 알고 있다고 가정하고 Feed Forward 함수를 직접 구현  
- 다음의 선형회귀 문제를 pytorch 로 해결  

$$b=-1,w=2$$

$$\hat{y}=-1+2x$$

In [3]:
w = torch.tensor(2.0)
b = torch.tensor(-1.0)
print(w, b)

tensor(2.) tensor(-1.)


In [4]:
def forward(x):
    yhat = w * x + b
    return yhat

x = 1 일 때 `y = 2x - 1`이므로 y 는 1

In [5]:
x = torch.tensor([1.0])
yhat = forward(x)

print(f"x = {x.item()} 일 때, y = {yhat.item()}")

x = 1.0 일 때, y = 1.0


x 가 multiple inputs (array) 인 경우:

In [6]:
x = torch.tensor([[1.0], [2.0], [3.0]])
forward(x)

tensor([[1.],
        [3.],
        [5.]])

In [7]:
y_hat = forward(x)
y_hat

tensor([[1.],
        [3.],
        [5.]])

<h1>2) torch.nn 의 Linear 클래스를 이용한 Linear Regression</h1>

In [8]:
from torch.nn import Linear
torch.manual_seed(1)

# 선형 회귀 모델 생성
# 입력 특성의 수 1, 출력 특성의 수 1
# bias=True: 모델에 편향(bias) 값을 포함
model = Linear(in_features=1, out_features=1, bias=True)

<code>state_dict()</code> method 는 각 layer 의 parameter tensor 를 dictionary 로 반환하므로 직접 값을 update 할 수 있다.

In [9]:
print(model.state_dict())
print()
print(model.state_dict()['weight'], model.state_dict()['bias'])
print()
print(model.weight)
print(model.bias)

OrderedDict([('weight', tensor([[0.5153]])), ('bias', tensor([-0.4414]))])

tensor([[0.5153]]) tensor([-0.4414])

Parameter containing:
tensor([[0.5153]], requires_grad=True)
Parameter containing:
tensor([-0.4414], requires_grad=True)


In [10]:
model.state_dict()['weight'][0] = torch.tensor(2)
model.state_dict()['bias'][0]     = torch.tensor(-1)

In [11]:
model.state_dict()

OrderedDict([('weight', tensor([[2.]])), ('bias', tensor([-1.]))])

In [12]:
# x = [[1.0]]일 경우 prediction --> 1.0*2 - 1 = 1
x = torch.tensor([[1.0]])
yhat = model(x)
yhat.item()

1.0

In [13]:
# x 가 multiple inputs (array) 인 경우:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)
print(yhat.detach().numpy())

[[1.]
 [3.]
 [5.]]


<h1>3) 사용자 정의 Module 작성</h1>

In [17]:
from torch import nn

class LR(nn.Module):
    def __init__(self, input_size, output_size):
        super(LR, self).__init__()
        # 선형 레이어를 정의합니다. 입력 크기와 출력 크기를 인수로 받습니다.
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        # 입력 x를 선형 레이어에 통과시켜 출력값을 계산합니다.
        out = self.linear(x)
        return out  # 최종 출력값을 반환합니다.

In [18]:
# 입력 size 1, 출력 size 1
model = LR(1, 1)

model.state_dict()

OrderedDict([('linear.weight', tensor([[-0.9414]])),
             ('linear.bias', tensor([0.5997]))])

In [19]:
model.state_dict()['linear.weight'][0] = torch.tensor(2)
model.state_dict()['linear.bias'][0]     = torch.tensor(-1)

print(list(model.parameters()))
print()
print(model.linear)

[Parameter containing:
tensor([[2.]], requires_grad=True), Parameter containing:
tensor([-1.], requires_grad=True)]

Linear(in_features=1, out_features=1, bias=True)


In [20]:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)

# 계산 그래프로부터 텐서를 분리하여 그라디언트 계산이 필요 없는 텐서로 만듭니다
print(yhat.detach().numpy())

[[1.]
 [3.]
 [5.]]


In [21]:
list(model.parameters())

[Parameter containing:
 tensor([[2.]], requires_grad=True),
 Parameter containing:
 tensor([-1.], requires_grad=True)]

# Backpropagation 과 경사하강법 적용

- weight 와 bias를 스스로 학습
- (습도, 강수량, 최고온도, 최저온도)를 feature 로 입력 받아  (사과, 오렌지)의 수확량(label)을 선형회귀로 예측하는 model 작성

## Data

In [22]:
# Input (습도, 강수량, 최고온도, 최저온도)
inputs = np.array([
                   [73, 67, 43, 10],
                   [91, 88, 64, 5],
                   [87, 134, 58, 2],
                   [102, 43, 37, 4],
                   [69, 96, 70, 5]], dtype='float32')

In [23]:
# Targets - (apples, oranges) 수확량
targets = np.array([
                    [56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

- input 과 target을 tensor로 변환

In [24]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs.size())
print(targets.size())

torch.Size([5, 4])
torch.Size([5, 2])


# 1) torch.nn Linear Class 직접 사용

In [25]:
from torch.nn import Linear
from torch import optim

torch.manual_seed(1)

In [28]:
# 입력 특징의 크기가 4이고 출력 특징의 크기가 2인 선형 모델을 정의합니다.
model = nn.Linear(in_features=4, out_features=2)

In [29]:
# 손실 함수로 평균 제곱 오차(MSE)를 정의합니다.
criterion = nn.MSELoss()

# 모델의 매개변수를 최적화하기 위해 확률적 경사 하강법(SGD) 옵티마이저를 정의합니다.
# 학습률(lr)은 1e-4로 설정합니다.
optimizer = optim.SGD(model.parameters(), lr=1e-4)

In [30]:
for epoch in range(100):               # 총 100번의 에포크 동안 학습을 반복합니다.
    for x, y in zip(inputs, targets):  # 입력 데이터와 타겟 데이터를 쌍으로 묶어 순회합니다.
        yhat = model(x)                # 모델에 입력 x를 전달하여 예측값 yhat을 얻습니다.
        loss = criterion(yhat, y)      # 예측값 yhat과 실제값 y를 비교하여 손실(loss)을 계산합니다.
        loss.backward()                # 손실에 대한 그래디언트를 계산합니다.
        optimizer.step()               # 옵티마이저를 사용하여 모델의 가중치를 업데이트합니다.
        optimizer.zero_grad()          # 다음 반복을 위해 옵티마이저의 그래디언트를 초기화합니다.

In [31]:
# 모델에 전체 입력 데이터를 전달하여 예측값(preds)을 얻습니다.
preds = model(inputs)

# 예측값(preds)과 실제 타겟값(targets)을 비교하여 손실(loss)을 계산합니다.
loss = criterion(preds, targets)
print(loss)

tensor(7.0686, grad_fn=<MseLossBackward0>)


In [32]:
# Predictions & target 비교
print(targets.detach().numpy())
print()
print(preds.detach().numpy())

[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]

[[ 57.842937  69.6347  ]
 [ 85.32951  100.18176 ]
 [125.2741   131.84752 ]
 [ 24.323542  36.581146]
 [104.19539  118.78626 ]]


# 2) 사용자 정의 model 사용

- Linear Regression 사용자 정의 model

In [33]:
from torch import nn

# 선형 회귀 모델을 정의하는 클래스입니다.
class LinearReg(nn.Module):
    def __init__(self, input_size, output_size):
        # nn.Module의 생성자를 호출하여 초기화합니다.
        super(LinearReg, self).__init__()
        # 입력 크기와 출력 크기를 받아 선형 레이어(fc)를 정의합니다.
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        # 입력 데이터 x를 선형 레이어(fc)에 통과시켜 예측값 yhat을 계산합니다.
        yhat = self.fc(x)
        # 예측값 yhat을 반환합니다.
        return yhat

In [34]:
model = LinearReg(inputs.shape[1], targets.shape[1])

In [35]:
criterion = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=1e-4)

In [39]:
# 모델의 구조를 출력합니다. 여기에는 모델의 레이어와 각 레이어의 세부 정보가 포함됩니다.
print(model)

# 모델의 현재 상태를 딕셔너리 형태로 출력합니다.
# 여기에는 각 레이어의 가중치와 바이어스 값 등이 포함됩니다.
print(model.state_dict())

LinearReg(
  (fc): Linear(in_features=4, out_features=2, bias=True)
)
OrderedDict([('fc.weight', tensor([[-0.1862, -0.3020, -0.0838, -0.2157],
        [-0.1602,  0.0239,  0.2981,  0.2718]])), ('fc.bias', tensor([-0.4888,  0.3100]))])


In [38]:
# 옵티마이저의 현재 상태를 딕셔너리 형태로 반환합니다.
# 여기에는 각 매개변수의 상태 및 옵티마이저 설정(예: 학습률, 모멘텀 등)이 포함됩니다.
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.0001,
   'momentum': 0,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'maximize': False,
   'foreach': None,
   'differentiable': False,
   'params': [0, 1]}]}

## Train the model

In [40]:
for epoch in range(100):
    for x, y in zip(inputs, targets):
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [41]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3504,  0.9005,  0.6323, -0.3524],
         [-0.3409,  0.7470,  0.9696,  0.2570]], requires_grad=True),
 Parameter containing:
 tensor([-0.4880,  0.3092], requires_grad=True)]

In [42]:
model.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.3504,  0.9005,  0.6323, -0.3524],
                      [-0.3409,  0.7470,  0.9696,  0.2570]])),
             ('fc.bias', tensor([-0.4880,  0.3092]))])

In [43]:
preds = model(inputs)

In [44]:
# Predictions & target 비교
print(targets.detach().numpy())
print()
print(preds.detach().numpy())

[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]

[[ 57.926044  69.7348  ]
 [ 85.56749   98.36236 ]
 [125.65458  127.49921 ]
 [ 24.472986  34.56048 ]
 [104.27415  117.65631 ]]


## Save and Load Model

In [45]:
# model save
torch.save(model.state_dict(), 'model.pt')

model.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.3504,  0.9005,  0.6323, -0.3524],
                      [-0.3409,  0.7470,  0.9696,  0.2570]])),
             ('fc.bias', tensor([-0.4880,  0.3092]))])

- model reload 및 parameter 복원 확인

In [46]:
reloaded = LinearReg(inputs.shape[1], targets.shape[1])
reloaded.load_state_dict(torch.load('model.pt'))

reloaded.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.3504,  0.9005,  0.6323, -0.3524],
                      [-0.3409,  0.7470,  0.9696,  0.2570]])),
             ('fc.bias', tensor([-0.4880,  0.3092]))])

In [47]:
preds = reloaded(inputs)

In [48]:
# Predictions & target 비교
print(targets.detach().numpy())
print()
print(preds.detach().numpy())

[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]

[[ 57.926044  69.7348  ]
 [ 85.56749   98.36236 ]
 [125.65458  127.49921 ]
 [ 24.472986  34.56048 ]
 [104.27415  117.65631 ]]
